In [19]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import shutil

In [20]:
def get_image_creation_date(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if exif_data is not None:
            for tag, value in exif_data.items():
                if TAGS.get(tag) == 'DateTimeOriginal':
                    return value.split(" ")[0] # nur das Datum zurückgeben
    except (IOError, AttributeError):
        pass

    return None



In [21]:
def get_list_of_creation_date(image_folder):
    images= []
    
    for filename in os.listdir(image_folder):
        if filename != ".DS_Store":
            image_path = os.path.join(image_folder, filename)
            with Image.open(image_path) as img:
                creation_date = img._getexif()[36867]
                images.append((image_path, creation_date))
    return images

In [22]:
def rename_and_resize_files(images, sorted_images, out_path, max_dim, team):
    counter = 0
    for index, (image_path,creation_date) in enumerate(sorted_images):
        
        date = creation_date.split(" ")[0]
        date = date.replace(':','')
        new_filename = f"{date}_{team}_{str(index).zfill(3)}.jpg"
        new_filepath = os.path.join(out_path, new_filename)
        
        with Image.open(image_path) as img:
            img.thumbnail((max_dim, max_dim), Image.ANTIALIAS)
            img.save(new_filepath)
            print(f"Renamed and resized: {new_filepath}")

In [23]:
directory = '/Users/sandra/Downloads/Kamera/'
images = get_list_of_creation_date(directory)

sorted_images = sorted(images, key=lambda x: x[1])

output_folder = '/Users/sandra/Documents/GitHub/sebert23.github.io/images/2023/07/'
rename_and_resize_files(images, sorted_images, output_folder, 1200, 'A-Junioren')
#rename_and_resize_files(directory, out_path, 1200, 'A-Junioren_Kobra_Abschluss')

/Users/sandra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  # This is added back by InteractiveShellApp.init_path()


Renamed and resized: /Users/sandra/Documents/GitHub/sebert23.github.io/images/2023/07/20230719_A-Junioren_000.jpg
Renamed and resized: /Users/sandra/Documents/GitHub/sebert23.github.io/images/2023/07/20230719_A-Junioren_001.jpg


In [3]:
# Beispielaufruf
image_path = "/Users/sandra/Documents/GitHub/sebert23.github.io/images/2023/06/"
creation_date = get_image_creation_date(image_path)

if creation_date is not None:
    print("Erstellungsdatum:", creation_date)
else:
    print("Erstellungsdatum nicht verfügbar oder Bild hat kein Erstellungsdatum.")


Erstellungsdatum: 2023:04:29 19:49:19


In [7]:
import os
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [8]:
def download_photos_from_google_folder(folder_id, save_dir):
    #Pfad erstellen um heruntergeladene Fotos zu speichern
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
        
    # Google Photos API-Verbindung herstellen
    credentials = service_account.Credentials.from_service_account_file('credentials.json')
    service = build('photoslibrary', 'v1', credentials=credentials, cache_discovery=False)

    # Fotos aus dem angegebenen Ordner abrufen
    results = service.mediaItems().search(body={'albumId': folder_id}).execute()
    photos = results.get('mediaItems', [])
    
    # Fotos herunterladen
    for photo in photos:
        photo_url = photo['baseUrl'] + '=d'
        file_name = photo['filename']
        save_path = os.path.join(save_dir, file_name)
        
        response = service._http.request('GET', photo_url)
        with open(save_path, 'wb') as f:
            f.write(response.data)
            
        print(f'{file_name} wurde heruntergeladen.')

In [11]:
import gdown
save_dir = "/Users/sandra/Documents/Test_Download/"
url= f"https://drive.google.com/uc?id={avysr5uUrKsWrUjb7}&export=download"
gdown.download(url, output=save_dir, quiet=False)

NameError: name 'avysr5uUrKsWrUjb7' is not defined

In [9]:
folder_id ="https://photos.google.com/u/1/album/AF1QipNLWawxiQbUlceavLDhDCR6FulXkbraZBRTlyNS"
save_dir = "/Users/sandra/Documents/Test_Download/"
download_photos_from_google_folder(folder_id, save_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'